In [1]:
import os, sys
import pandas as pd
import numpy as np
#readind and preparing data ( text , label )
data=pd.read_excel('ml2.xlsx', header=None) ## read le file
data.columns=['text','label']
print(data[1:5])

                                                text  label
1  محضر تحقيق بالشكوى المقدمة بنك الشرق ضد جوزف ع...     12
2      حول ادعاء طوني يارد ضد مجهول بجرم سرقة محفظته     13
3  محضر تحقيق فوري بتوقيف اللبناني فارس ورده بجرم...     11
4  محضر تحقيق بإدعاء حسام حسين قبيسي ضد المدعى عل...     12


In [3]:
Xt=data['text']
#X_train=Xt[0:5000]
#print(X_train[1])
Yt=data['label']
#Y_train=Yt[0:5000]
##print(Y_train[1])
#X_test=Xt[5001:]
#Y_test=Yt[5001:]


In [4]:
# -*- coding: utf-8 -*-

# Arabic Transliteration based on Buckwalter
# dictionary source is buckwalter2unicode.py http://www.redhat.com/archives/fedora-extras-commits/2007-June/msg03617.html 

buck2uni = {"'": u"\u0621", # hamza-on-the-line
            "|": u"\u0622", # madda
            ">": u"\u0623", # hamza-on-'alif
            "&": u"\u0624", # hamza-on-waaw
            "<": u"\u0625", # hamza-under-'alif
            "}": u"\u0626", # hamza-on-yaa'
            "A": u"\u0627", # bare 'alif
            "b": u"\u0628", # baa'
            "p": u"\u0629", # taa' marbuuTa
            "t": u"\u062A", # taa'
            "v": u"\u062B", # thaa'
            "j": u"\u062C", # jiim
            "H": u"\u062D", # Haa'
            "x": u"\u062E", # khaa'
            "d": u"\u062F", # daal
            "*": u"\u0630", # dhaal
            "r": u"\u0631", # raa'
            "z": u"\u0632", # zaay
            "s": u"\u0633", # siin
            "$": u"\u0634", # shiin
            "S": u"\u0635", # Saad
            "D": u"\u0636", # Daad
            "T": u"\u0637", # Taa'
            "Z": u"\u0638", # Zaa' (DHaa')
            "E": u"\u0639", # cayn
            "g": u"\u063A", # ghayn
            "_": u"\u0640", # taTwiil
            "f": u"\u0641", # faa'
            "q": u"\u0642", # qaaf
            "k": u"\u0643", # kaaf
            "l": u"\u0644", # laam
            "m": u"\u0645", # miim
            "n": u"\u0646", # nuun
            "h": u"\u0647", # haa'
            "w": u"\u0648", # waaw
            "Y": u"\u0649", # 'alif maqSuura
            "y": u"\u064A", # yaa'
            "F": u"\u064B", # fatHatayn
            "N": u"\u064C", # Dammatayn
            "K": u"\u064D", # kasratayn
            "a": u"\u064E", # fatHa
            "u": u"\u064F", # Damma
            "i": u"\u0650", # kasra
            "~": u"\u0651", # shaddah
            "o": u"\u0652", # sukuun
            "`": u"\u0670", # dagger 'alif
            "{": u"\u0671", # waSla
}
def transString(string, reverse=0):
    '''Given a Unicode string, transliterate into Buckwalter. To go from
    Buckwalter back to Unicode, set reverse=1'''

    for k, v in buck2uni.items():
      if not reverse:
            string = string.replace(v, k)
      else:
            string = string.replace(k, v)

    return string


In [5]:
for i in range(len(Xt)):
    Xt[i]=transString(Xt[i])


C:\Users\Maya\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
len(Xt)
print(data['text'][8])
print(Xt[8])

mHDr tHqyq bAdEA' fwAz mHmd Abw Emr/ lbnAny Dd mjhwlyn bjrm srqp bwAsTp Alksr wAlxlE. 
mHDr tHqyq bAdEA' fwAz mHmd Abw Emr/ lbnAny Dd mjhwlyn bjrm srqp bwAsTp Alksr wAlxlE. 


In [11]:
from snowballstemmer import stemmer
import nltk
nltk.download('stopwords')
SW=set(stopwords.words('english'))
#ar_stemmer = stemmer("arabic")
#ar_stemmer.stemWord(u"فسميتموها")
def prep(doc):
    words=str(doc).strip().split()###  strip pr eliminer les white spaces, lower por transformer en miniscules
    #stemmer=SnowballStemmer("arabic")
    #ar_stemmer = stemmer("arabic")
    #words=ar_stemmer.stem(w.decode('UTF-8'))
    words=[stemmer.stem(w) for w in words  if not w in SW]## transformer en stem without les stopwords
    return ' '.join(words)## met des spaces entre les mots
xt=Xt.apply(prep)#### select le colomn text selement et la preper(prep)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'stopwords' is not defined

In [15]:
def prep(doc):
    words=str(doc).strip().split()
    return ''.join(words)
XT=Xt.apply(prep)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
model1=TfidfVectorizer()
tfidf_matrix=model1.fit_transform(XT)

In [18]:
from sklearn.model_selection import train_test_split

labels=data['label']
train,test,ltrain,ltest =\
train_test_split(tfidf_matrix, labels,test_size=0.2)


In [20]:

from sklearn.neural_network import MLPClassifier

mlp=MLPClassifier(hidden_layer_sizes=(50,30,), max_iter=120, solver='adam',verbose =10 , tol=1e-4, random_state=2, learning_rate_init=0.1)
model=mlp.fit(train.todense(),ltrain)

Iteration 1, loss = 2.10071537
Iteration 2, loss = 1.41568510
Iteration 3, loss = 0.87649561
Iteration 4, loss = 0.54118995
Iteration 5, loss = 0.36983646
Iteration 6, loss = 0.27539956
Iteration 7, loss = 0.19484270
Iteration 8, loss = 0.16269285
Iteration 9, loss = 0.16689769
Iteration 10, loss = 0.15249663
Iteration 11, loss = 0.14887979
Iteration 12, loss = 0.13974364
Iteration 13, loss = 0.13645701
Iteration 14, loss = 0.12231139
Iteration 15, loss = 0.12030324
Iteration 16, loss = 0.13419733
Iteration 17, loss = 0.14809884
Iteration 18, loss = 0.21203003
Iteration 19, loss = 0.26580160
Iteration 20, loss = 0.27561326
Iteration 21, loss = 0.31295914
Iteration 22, loss = 0.27506503
Iteration 23, loss = 0.25585115
Iteration 24, loss = 0.20318597
Iteration 25, loss = 0.18493851
Iteration 26, loss = 0.18097808
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


In [21]:
from sklearn import metrics
prediction=mlp.predict(test)
cm=metrics.confusion_matrix(ltest,prediction)
print('confusion matrix is:')
print(cm)
print()
print('classification_report is:')
print(metrics.classification_report(ltest, prediction))#ngram(1,2 without to dense)
print("train Accuracy: %f" %mlp.score(train,ltrain))
print("test Accuracy: %f" %mlp.score(test,ltest))

confusion matrix is:
[[ 25   0   1   0   0   2  11  14   3   4   0   0   5]
 [ 10   1   1   1   2   9   0   0  15   9   0   0  10]
 [ 10   2  33   0   0   3   0   0   9  12   3   0   5]
 [  9   1   2  15   1  11   0   0  10  14   0   0  10]
 [  7   2   0   1  12  26   0   0  14   9   0   6  17]
 [  9   1   0   8   4  17   0   2  20  15   2   1  13]
 [ 45   0   1   0   0   0  23  32   0   1   0   0   0]
 [ 60   0   0   0   0   0   0  47   0   0   0   0   0]
 [ 13   0   2   0   0   3   0   1  65  11   0   0  12]
 [ 19   0   2   7   0   2   0   2  23  42   2   0  24]
 [125   0   1   3   0   1   0  10   0   4  15   0   1]
 [ 14   4   2   0   5   7   0   0   3   2   0 155   5]
 [ 31   9   5   1   2  28   0   1  50  24   3   3  37]]

classification_report is:
              precision    recall  f1-score   support

           1       0.07      0.38      0.11        65
           2       0.05      0.02      0.03        58
           3       0.66      0.43      0.52        77
           4       

In [22]:
#ngram(1,2 without to dense)
print("train Accuracy: %f" %mlp.score(train,ltrain))
print("test Accuracy: %f" %mlp.score(test,ltest))

train Accuracy: 0.975833
test Accuracy: 0.336094


In [28]:
#import os, sys
#import pandas as pd
#import numpy as np
#from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import BernoulliNB
#from sklearn.ensemble import RandomForestClassifier
##from sklearn.metrics import roc_curve, auc
#from sklearn import svm
#from sklearn.metrics import classification_report, confusion_matrix
#from sklearn.model_selection import GridSearchCV
from gensim.models import KeyedVectors
#from scipy import spatial
#import matplotlib.pyplot as plt
#from sklearn.neural_network import MLPClassifier
from nltk import bigrams
import nltk
import re
#import spacy
###FUNCTIONS
def avg_feature_vector(sentence, model, num_features, index2word_set):
        words = sentence.split()
        feature_vec = np.zeros((num_features, ), dtype='float32')
        n_words = 0
        for word in words:
            if word in index2word_set:
                n_words += 1
                feature_vec = np.add(feature_vec, model[word])
        if (n_words > 0):
            feature_vec = np.divide(feature_vec, n_words)
        return feature_vec

def convert_word2vec(sentences,num_features):
        vectors_matrix = np.zeros((len(sentences),num_features))
        i = 0
        for sent in sentences:
            temp = avg_feature_vector(sent,word2vec_vectors,num_features,index2word_set)
            vectors_matrix[i,:] = temp
            i = i+1
        return vectors_matrix
def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token for token in doc ]
    #words=str(doc).strip().split()
    #words=[st.stem(w) for w in words ]
    #return ' '.join(words)
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
      return ' '.join(txt)
#from gensim.models.phrases import Phrases, Phraser
#data=pd.read_excel('ml2.xlsx', header=None) ## read le file
#data.columns=['text','label']
#def prep(doc):
 #   words=str(doc).strip().split()
  #  return ' '.join(words)
#Xt=Xt.apply(prep)
#sentences = Xt
data=Xt
#data = data[data['Semantic_Type']!='DISREGARD']
#from gensim.models.phrases import Phrases, Phraser
#from nltk.stem import ISRIStemmer
#st=ISRIStemmer()
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in data)
txt = [cleaning(doc) for doc in data]
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape
sent = [row.split() for row in df_clean['clean']]
sentences = sent
from gensim.models import Word2Vec
word2vec_vectors = Word2Vec(min_count=2,window=5,size=200,sample=6e-5,alpha=0.03,min_alpha=0.0007,negative=20)
word2vec_vectors.build_vocab(sentences)
word2vec_vectors.train(sentences, total_examples=word2vec_vectors.corpus_count, epochs=30, report_delay=1)
word2vec_vectors.init_sims(replace=True)
index2word_set = set(word2vec_vectors.wv.index2word)
###vectorizing the sentences
#vectors_matrix = convert_word2vec(data['Sentence'],200)
#np.random.seed(10)
#labels=data['label']
#train,test,ltrain,ltest =train_test_split(vectors_matrix, labels,test_size=0.2)#,random_state=8)

C:\Users\Maya\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [29]:
vectors_matrix = convert_word2vec(Xt,200)
np.random.seed(10)
#labels=Yt
#train,test,ltrain,ltest =train_test_split(vectors_matrix, labels,test_size=0.2)#,random_state=8)

C:\Users\Maya\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [31]:
print(vectors_matrix[10])

[ 3.19547369e-03  5.82107157e-03 -3.37295979e-02  2.58115376e-03
 -5.45239747e-02 -1.11295097e-02  2.23629288e-02 -9.70533639e-02
  5.43381870e-02  1.15961693e-01 -1.67765290e-01  6.15069270e-02
  9.10789445e-02 -6.18498735e-02  1.28274396e-01 -1.32828997e-02
 -4.23808135e-02  5.98876849e-02 -1.68288082e-01 -6.40497496e-03
  4.03265283e-02 -5.34946984e-03 -1.20958947e-02  1.27212465e-01
 -2.70150192e-02  5.80591895e-03 -8.85659307e-02  3.66403833e-02
  7.43004084e-02  2.79699396e-02 -6.99312687e-02  7.49559104e-02
 -2.35138517e-02 -1.46704212e-01  7.79712871e-02  3.86479683e-02
 -9.15474743e-02  1.18426129e-03  1.00725638e-02  1.45641640e-01
  9.51596797e-02  1.49067804e-01 -4.20593619e-02 -1.46789134e-01
  1.11378260e-01  4.12388965e-02 -1.62494816e-02  6.38975054e-02
 -1.28784731e-01  3.79103459e-02 -4.01398055e-02  1.58800872e-03
  1.82550922e-02 -1.32319018e-01 -4.35764156e-02  9.50044841e-02
 -1.70010403e-01 -2.86953803e-02  2.02054717e-02  2.56861318e-02
 -1.01169966e-01 -4.16843

In [32]:
X_train=vectors_matrix[10:5000]
#print(X_train[1])
Y_train=Yt[10:5000]
#print(Y_train[10])
X_test=vectors_matrix[5001:]
Y_test=Yt[5001:]
#print(X_test[10])
#print(Y_test[5001])

In [33]:
import tensorflow as tf
import tensorflow.python.framework
from tensorflow.python.framework import ops
import tensorflow.keras.backend
import numpy
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.preprocessing import sequence
from keras.utils import to_categorical
#X_train=train
#Y_train=ltrain
#X_test=test
#Y_test=ltest
num_classes = 14
max_mahdar_length = 100
#sequences=train
X_train = sequence.pad_sequences(X_train, maxlen=max_mahdar_length) #, truncatng)
X_test = sequence.pad_sequences(X_test, maxlen=max_mahdar_length) #,dtype='int32')
Y_train = to_categorical(Y_train, num_classes)# mn7awel l y la one hot unicoding yaani vecteur kl chi zero ma 3ada class tab3ha bi7eta 1
Y_test  = keras.utils.to_categorical(Y_test, num_classes)
top_words=5000
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_mahdar_length))
model.add(Conv1D(filters=16, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(50))
model.add(Dense(32,activation='relu'))# yaani 128 neurones
#model.add(Dropout(0.25))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy , optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 32)           160000    
_________________________________________________________________
conv1d (Conv1D)              (None, 100, 16)           2576      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 50, 16)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                13400     
_________________________________________________________________
dense (Dense)                (None, 32)                1632      
_________________________________________________________________
dense_1 (Dense)              (None, 14)                462       
Total params: 178,070
Trainable params: 178,070
Non-trainable params: 

In [34]:
from sklearn.neural_network import MLPClassifier

mlp=MLPClassifier(hidden_layer_sizes=(50,30,), max_iter=150, solver='adam',verbose =10 , tol=1e-4, random_state=2, learning_rate_init=0.01)
model=mlp.fit(X_train,Y_train)

Iteration 1, loss = 5.99526468
Iteration 2, loss = 3.52027014
Iteration 3, loss = 3.42364991
Iteration 4, loss = 3.41318532
Iteration 5, loss = 3.41181890
Iteration 6, loss = 3.41246669
Iteration 7, loss = 3.41228931
Iteration 8, loss = 3.41160408
Iteration 9, loss = 3.41341597
Iteration 10, loss = 3.41594193
Iteration 11, loss = 3.41302295
Iteration 12, loss = 3.41518361
Iteration 13, loss = 3.41164742
Iteration 14, loss = 3.41271271
Iteration 15, loss = 3.41397465
Iteration 16, loss = 3.41232506
Iteration 17, loss = 3.41056363
Iteration 18, loss = 3.41301879
Iteration 19, loss = 3.41226056
Iteration 20, loss = 3.41210582
Iteration 21, loss = 3.41297397
Iteration 22, loss = 3.41147127
Iteration 23, loss = 3.41459637
Iteration 24, loss = 3.41107278
Iteration 25, loss = 3.41027125
Iteration 26, loss = 3.41326955
Iteration 27, loss = 3.41339301
Iteration 28, loss = 3.41226830
Iteration 29, loss = 3.41359353
Iteration 30, loss = 3.41111852
Iteration 31, loss = 3.41172452
Iteration 32, los

In [35]:
print("train Accuracy: %f" %mlp.score(X_train,Y_train))
print("test Accuracy: %f" %mlp.score(X_test,Y_test))

train Accuracy: 0.000000
test Accuracy: 0.000000
